# HR Interview Orchestrator Demo 🎯

**AI-Powered Candidate Screening & Interview Automation**

This notebook demonstrates the key features of the HR Interview Orchestrator system:
- Job description parsing and validation
- Candidate resume analysis
- Interview question generation
- Complete hiring workflow automation

---


## 🚀 Setup and Imports


In [ ]:
import sys
import json
from pathlib import Path
import pandas as pd
from datetime import datetime

# Add parent directory to path for imports
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

try:
    # Import HR Orchestrator components
    from src.tools.mcp_tool import (
        parse_job_description_internal, 
        validate_job_description_quality,
        job_description_tool
    )
    from src.tools.parser import parse_resume_to_struct, load_text
    from src.agents import _score_candidate
    from src.state import JD, Candidate
    from src.tools.retriever import LocalQuestionBank
    from src.config import config

    print("✅ HR Interview Orchestrator components loaded successfully!")
    print(f"📁 Working directory: {Path.cwd()}")
    print(f"🏢 Company: {config.COMPANY_NAME}")
    
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("💡 This notebook should be run from the 'notebooks' directory")
    print("📁 Make sure the parent directory contains the 'src' folder")
    print("🔧 You may need to install dependencies: pip install -r requirements.txt")
    
    # Show current directory structure for debugging
    print(f"\n📋 Current directory: {Path.cwd()}")
    print(f"📋 Parent directory: {parent_dir}")
    print(f"📋 Looking for: {parent_dir / 'src'}")
    print(f"📋 Src exists: {(parent_dir / 'src').exists()}")
    
    raise


## 📄 Job Description Analysis

Let's start by analyzing a job description using our internal MCP tool.


In [ ]:
# Sample job description
sample_jd = """
# Senior Full Stack Developer

**Location:** San Francisco, CA / Remote

**About the Role:**
We're looking for an experienced Full Stack Developer to join our growing team.

**Requirements:**
- 5+ years of React and TypeScript experience
- Strong Node.js and Express.js background
- Experience with PostgreSQL or MongoDB
- Docker and AWS deployment experience
- Test-driven development (Jest, Cypress)

**Nice to Have:**
- GraphQL experience
- Kubernetes knowledge
- Microservices architecture
- Team leadership experience
"""

print("🔍 Analyzing Job Description...")
print("=" * 40)

# Parse the job description
jd_data = parse_job_description_internal(sample_jd, include_metadata=True)

print(f"📋 Title: {jd_data['title']}")
print(f"📍 Location: {jd_data['location']}")
print(f"✅ Must-haves: {len(jd_data['must_haves'])} requirements")
print(f"⭐ Nice-haves: {len(jd_data['nice_haves'])} preferences")
print(f"📊 Status: {jd_data['parsing_status']}")

# Display requirements
print("\n🔧 Required Skills:")
for i, skill in enumerate(jd_data['must_haves'], 1):
    print(f"  {i}. {skill}")

print("\n⭐ Preferred Skills:")
for i, skill in enumerate(jd_data['nice_haves'], 1):
    print(f"  {i}. {skill}")


## 📊 Job Description Quality Assessment


In [ ]:
# Validate job description quality
quality_result = validate_job_description_quality(sample_jd)

print("📊 Job Description Quality Assessment")
print("=" * 45)
print(f"🎯 Overall Score: {quality_result['quality_score']}/100")
print(f"🏆 Quality Tier: {quality_result['quality_tier']}")
print(f"📋 Requirements Found: {quality_result['requirements_found']}")
print(f"⭐ Preferences Found: {quality_result['preferences_found']}")
print(f"📝 Has Title: {quality_result['has_title']}")
print(f"📍 Has Location: {quality_result['has_location']}")

if quality_result['issues']:
    print("\n⚠️ Issues Found:")
    for issue in quality_result['issues']:
        print(f"  • {issue}")

if quality_result['suggestions']:
    print("\n💡 Suggestions:")
    for suggestion in quality_result['suggestions']:
        print(f"  • {suggestion}")

# Create a quality breakdown chart
quality_breakdown = {
    'Title': 20 if quality_result['has_title'] else 0,
    'Location': 10 if quality_result['has_location'] else 0,
    'Requirements': min(40, quality_result['requirements_found'] * 13.33),
    'Preferences': min(15, quality_result['preferences_found'] * 7.5),
    'Content Detail': quality_result['quality_score'] - sum([
        20 if quality_result['has_title'] else 0,
        10 if quality_result['has_location'] else 0,
        min(40, quality_result['requirements_found'] * 13.33),
        min(15, quality_result['preferences_found'] * 7.5)
    ])
}

print("\n📈 Quality Breakdown:")
for category, score in quality_breakdown.items():
    bar = "█" * int(score / 5) + "░" * (20 - int(score / 5))
    print(f"  {category:15} {bar} {score:5.1f}/100")


## 👥 Candidate Resume Analysis

Now let's analyze some candidate resumes and score them against our job requirements.


In [ ]:
# Sample candidate resumes
candidates_data = [
    {
        "name": "Alice Johnson",
        "resume": """
        Alice Johnson
        Senior Software Engineer
        alice.johnson@email.com
        
        Experience:
        • 6 years React and TypeScript development
        • 4 years Node.js and Express.js backend
        • PostgreSQL and MongoDB experience
        • AWS deployment and Docker containerization
        • Jest and Cypress testing frameworks
        • GraphQL API development
        • Led team of 3 developers
        """
    },
    {
        "name": "Bob Smith",
        "resume": """
        Bob Smith
        Full Stack Developer
        bob.smith@email.com
        
        Experience:
        • 3 years React development
        • 2 years Node.js experience
        • MySQL database experience
        • Basic AWS knowledge
        • Unit testing with Jest
        • Python and Django background
        """
    },
    {
        "name": "Carol Davis",
        "resume": """
        Carol Davis
        Frontend Developer
        carol.davis@email.com
        
        Experience:
        • 7 years React and TypeScript
        • 5 years Node.js and Express
        • PostgreSQL and Redis experience
        • Docker and Kubernetes deployment
        • Comprehensive testing (Jest, Cypress, Playwright)
        • GraphQL and REST API development
        • Microservices architecture
        • Team lead for 5+ developers
        """
    }
]

# Create JD object for scoring
jd = JD(
    title=jd_data['title'],
    location=jd_data['location'],
    must_haves=jd_data['must_haves'],
    nice_haves=jd_data['nice_haves']
)

print("👥 Analyzing Candidates...")
print("=" * 30)

analyzed_candidates = []

for candidate_data in candidates_data:
    # Parse resume
    candidate = parse_resume_to_struct(candidate_data['resume'], f"{candidate_data['name']}.txt")
    
    # Score candidate
    score = _score_candidate(jd, candidate)
    candidate.score = score
    
    analyzed_candidates.append(candidate)
    
    print(f"\n📋 {candidate.name}")
    print(f"   📧 Email: {candidate.email}")
    print(f"   🎯 Score: {score:.3f} ({score*100:.1f}%)")
    print(f"   💼 Experience: {candidate.years_exp} years")
    print(f"   🔧 Skills: {len(candidate.skills)} identified")
    print(f"   🏆 Top Skills: {', '.join(candidate.skills[:5])}")

print(f"\n✅ Analyzed {len(analyzed_candidates)} candidates successfully!")


## 🏆 Candidate Ranking and Comparison


In [ ]:
# Sort candidates by score
ranked_candidates = sorted(analyzed_candidates, key=lambda x: x.score, reverse=True)

print("🏆 Candidate Rankings")
print("=" * 25)

# Create a comparison DataFrame
comparison_data = []
for i, candidate in enumerate(ranked_candidates, 1):
    # Calculate skill overlap
    must_have_matches = len(set(jd.must_haves) & set(candidate.skills))
    nice_have_matches = len(set(jd.nice_haves) & set(candidate.skills))
    
    # Determine status
    if candidate.score >= 0.7:
        status = "🟢 Excellent"
    elif candidate.score >= 0.5:
        status = "🟡 Good"
    elif candidate.score >= 0.3:
        status = "🟠 Fair"
    else:
        status = "🔴 Poor"
    
    comparison_data.append({
        'Rank': i,
        'Name': candidate.name,
        'Score': f"{candidate.score:.3f}",
        'Match %': f"{candidate.score*100:.1f}%",
        'Experience': f"{candidate.years_exp}y",
        'Must-Have Matches': f"{must_have_matches}/{len(jd.must_haves)}",
        'Nice-Have Matches': f"{nice_have_matches}/{len(jd.nice_haves)}",
        'Status': status
    })

# Display as DataFrame
df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

# Show detailed skill analysis for top candidate
top_candidate = ranked_candidates[0]
print(f"\n🎯 Top Candidate: {top_candidate.name}")
print("=" * 35)

must_have_skills = set(jd.must_haves)
candidate_skills = set(top_candidate.skills)
matched_skills = must_have_skills & candidate_skills
missing_skills = must_have_skills - candidate_skills

print(f"✅ Matched Skills ({len(matched_skills)}/{len(must_have_skills)}):")
for skill in matched_skills:
    print(f"   • {skill}")

if missing_skills:
    print(f"\n❌ Missing Skills ({len(missing_skills)}):")
    for skill in missing_skills:
        print(f"   • {skill}")

# Show bonus skills
nice_have_skills = set(jd.nice_haves)
bonus_skills = candidate_skills & nice_have_skills
if bonus_skills:
    print(f"\n⭐ Bonus Skills ({len(bonus_skills)}):")
    for skill in bonus_skills:
        print(f"   • {skill}")


## ❓ Interview Question Generation

Let's generate tailored interview questions for our top candidates.


In [ ]:
# Initialize question bank
try:
    qbank = LocalQuestionBank("../data/question_bank.csv")
    
    # Generate search query from job requirements
    search_query = " ".join(jd.must_haves[:5])  # Top 5 skills
    
    print("❓ Generating Interview Questions")
    print("=" * 35)
    print(f"🔍 Search Query: {search_query}")
    
    # Get relevant questions
    questions = qbank.get_relevant_questions(search_query, top_k=8)
    
    print(f"\n📋 Generated {len(questions)} Questions:")
    print("=" * 40)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. {question}")
    
    # Categorize questions by type
    technical_keywords = ['implement', 'design', 'architecture', 'performance', 'optimize']
    behavioral_keywords = ['experience', 'challenge', 'team', 'project', 'approach']
    
    technical_questions = [q for q in questions if any(kw in q.lower() for kw in technical_keywords)]
    behavioral_questions = [q for q in questions if any(kw in q.lower() for kw in behavioral_keywords)]
    
    print(f"\n📊 Question Breakdown:")
    print(f"   🔧 Technical: {len(technical_questions)} questions")
    print(f"   👥 Behavioral: {len(behavioral_questions)} questions")
    print(f"   📋 Other: {len(questions) - len(technical_questions) - len(behavioral_questions)} questions")
    
except Exception as e:
    print(f"⚠️ Could not load question bank: {e}")
    print("💡 Using sample questions instead...")
    
    # Fallback sample questions
    sample_questions = [
        "How would you optimize a React application for better performance?",
        "Explain your approach to error handling in Node.js applications.",
        "Describe your experience with TypeScript and its benefits.",
        "How do you ensure code quality in a team environment?",
        "Walk me through your process for debugging a production issue.",
        "What's your experience with database optimization?",
        "How do you approach testing in full-stack applications?",
        "Describe a challenging technical problem you solved recently."
    ]
    
    print(f"\n📋 Sample Questions ({len(sample_questions)}):")
    for i, question in enumerate(sample_questions, 1):
        print(f"\n{i}. {question}")


## 📊 Complete Hiring Pipeline Summary


In [ ]:
# Generate comprehensive hiring summary
print("📊 Hiring Pipeline Summary")
print("=" * 30)

# Job summary
print(f"\n📋 Position: {jd.title}")
print(f"📍 Location: {jd.location}")
print(f"🎯 Quality Score: {quality_result['quality_score']}/100 ({quality_result['quality_tier']})")

# Candidate summary
total_candidates = len(analyzed_candidates)
qualified_candidates = len([c for c in analyzed_candidates if c.score >= 0.5])
excellent_candidates = len([c for c in analyzed_candidates if c.score >= 0.7])

print(f"\n👥 Candidates Processed: {total_candidates}")
print(f"✅ Qualified (50%+): {qualified_candidates}")
print(f"🏆 Excellent (70%+): {excellent_candidates}")
print(f"📈 Success Rate: {qualified_candidates/total_candidates*100:.1f}%")

# Top candidates for interviews
interview_candidates = ranked_candidates[:2]  # Top 2
print(f"\n🎯 Recommended for Interview ({len(interview_candidates)}):")
for i, candidate in enumerate(interview_candidates, 1):
    print(f"   {i}. {candidate.name} - {candidate.score*100:.1f}% match")

# Process metrics
avg_score = sum(c.score for c in analyzed_candidates) / len(analyzed_candidates)
max_score = max(c.score for c in analyzed_candidates)
min_score = min(c.score for c in analyzed_candidates)

print(f"\n📈 Scoring Metrics:")
print(f"   📊 Average Score: {avg_score:.3f} ({avg_score*100:.1f}%)")
print(f"   🏆 Highest Score: {max_score:.3f} ({max_score*100:.1f}%)")
print(f"   📉 Lowest Score: {min_score:.3f} ({min_score*100:.1f}%)")

# Recommendations
print(f"\n💡 Recommendations:")
if excellent_candidates == 0:
    print("   • Consider widening search criteria or improving job posting")
if qualified_candidates < 2:
    print("   • Expand candidate sourcing efforts")
if avg_score < 0.4:
    print("   • Review job requirements - may be too restrictive")
else:
    print("   • Strong candidate pool - proceed with interviews")
    print(f"   • Schedule interviews with top {min(2, qualified_candidates)} candidates")
    print("   • Prepare technical and behavioral questions")

print(f"\n🎉 Analysis Complete! Ready to proceed with hiring process.")


## 🔧 MCP Tool Schema Export

For reference, here's the MCP tool schema that could be used for external integrations.


In [ ]:
# Display MCP tool schema
schema = job_description_tool.get_tool_schema()

print("🔧 MCP Tool Schema")
print("=" * 20)
print(json.dumps(schema, indent=2))

# Save schema to file for reference
try:
    schema_path = Path("../artifacts/mcp_tool_schema.json")
    schema_path.parent.mkdir(exist_ok=True)
    with open(schema_path, 'w') as f:
        json.dump(schema, f, indent=2)
    print(f"\n💾 Schema saved to: {schema_path}")
except Exception as e:
    print(f"\n⚠️ Could not save schema: {e}")

print("\n💡 This schema can be used for:")
print("   • External MCP server implementations")
print("   • API documentation")
print("   • Integration with AI tools (Claude Desktop, etc.)")
print("   • Custom workflow automation")


## 🎯 Next Steps

This demo showcased the core capabilities of the HR Interview Orchestrator:

### ✅ What We Accomplished
- **Job Description Analysis**: Parsed and validated JD quality
- **Candidate Evaluation**: Scored multiple candidates against requirements
- **Ranking & Comparison**: Identified top candidates for interviews
- **Question Generation**: Created relevant interview questions
- **Complete Pipeline**: End-to-end hiring workflow automation

### 🚀 Production Usage
To use the full system:

```bash
# Run complete pipeline
python -m src.graph --jd data/sample_jd.md --resumes data/resumes

# Parse JD only
python -m src.graph --jd data/sample_jd.md --resumes data/resumes --parse-jd-only

# Validate JD quality
python -m src.graph --jd data/sample_jd.md --resumes data/resumes --validate-jd
```

### 🔗 Integration Options
- **LangSmith**: Enable tracing with `LANGSMITH_TRACING=1`
- **Google Integration**: Connect Gmail and Calendar
- **Custom Workflows**: Use internal MCP tool for automation
- **API Integration**: Build custom interfaces using the core components

### 🔧 Internal MCP Tool
The notebook demonstrates the internal MCP tool that provides:
- Structured job description parsing
- Quality validation and scoring
- MCP-compliant schema for future integrations
- Standalone functionality for custom workflows

---

**🎉 Happy Hiring!** The HR Interview Orchestrator makes candidate screening efficient, consistent, and data-driven.
